In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Twitter US Airline Sentiment

# Problem Definition

the task is to predict whether a tweet contains positive, negative, or neutral sentiment about the airline. This is a typical supervised learning task where given a text string, we have to categorize the text string into predefined categories.
Data analysis on the dataset to find the best and the worst airlines and understand what are the most common problems in case of bad flight Training two Naive-Bayesian classifiers: first to classify the tweets into positive and negative And a second classifier to classify the negative tweets on the reason. In this way is possible to upgrade the statistics, of the first point, with new tweets.

# What is Sentiment Analysis?


Sentiment Analysis is the process of determining whether is positive, negative or neutral. It can be used to identify the customer or follower's attitude towards a brand through the use of variables such as context, tone, emotion Marketers can use sentiment analysis to research public opinion of their company and products, or to analyze customer satisfaction. Organizations can also use this analysis to gather critical feedback about problems in newly released products Sentiment analysis not only helps companies understand how they’re doing with their customers, it also gives them a better picture of how they stack up against their competitors.

# Importing the Libraries


In [ ]:
import numpy as np 
import pandas as pd 
import re
import nltk 
import matplotlib.pyplot as plt

# Importing the Dataset


In [ ]:
tweets = pd.read_csv('/kaggle/input/twitter-airline-sentiment/Tweets.csv')

In [ ]:
 tweets.head()

In [ ]:
tweets.info()

In [ ]:
tweets.shape

In [ ]:
tweets.airline.value_counts().plot(kind='pie', autopct='%1.0f%%')

In the output, you can see the percentage of public tweets for each airline. United Airline has the highest number of tweets i.e. 26%, followed by US Airways (20%)

see the distribution of sentiments across all the tweets.


In [ ]:
airline_sentiment = tweets.groupby(['airline', 'airline_sentiment']).airline_sentiment.count()
airline_sentiment.plot(kind='bar')

the majority of the tweets are negative, followed by neutral and positive tweets. 

# Data Cleaning

In [ ]:
features = tweets.iloc[:, 10].values
labels = tweets.iloc[:, 1].values

divide the data into features and training set, we can preprocess data in order to clean it

In [ ]:
processed_features = []

for sentence in range(0, len(features)):
    # Remove  special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))
    # remove single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 
    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)
    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)
    # Converting to Lowercase
    processed_feature = processed_feature.lower()
    processed_features.append(processed_feature)

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
processed_features = vectorizer.fit_transform(processed_features).toarray()

 # Filter for the category

In [ ]:
is_positive = tweets['airline_sentiment'].str.contains("positive")
is_negative = tweets['airline_sentiment'].str.contains("negative")
is_neutral = tweets['airline_sentiment'].str.contains("neutral")

In [ ]:
 positive_tweets = tweets[is_positive]
 positive_tweets.shape


In [ ]:
negative_tweets = tweets[is_negative]
negative_tweets.shape


In [ ]:
neutral_tweets = tweets[is_neutral]
neutral_tweets.shape

In [ ]:
 worst_airline = negative_tweets[['airline','airline_sentiment_confidence','negativereason']]
worst_airline   

# Create the rank for the worst airline


In [ ]:
cnt_worst_airline = worst_airline.groupby('airline', as_index=False).count()
cnt_worst_airline.sort_values('negativereason', ascending=False)

# Create the rank for the best airline


In [ ]:
best_airline = positive_tweets[['airline','airline_sentiment_confidence']]
cnt_best_airline = best_airline.groupby('airline', as_index=False).count()
cnt_best_airline.sort_values('airline_sentiment_confidence', ascending=False)


# Create the rank for negative reason


In [ ]:
motivation = negative_tweets[['airline','negativereason']]
cnt_bad_flight_motivation = motivation.groupby('negativereason', as_index=False).count()
cnt_bad_flight_motivation.sort_values('negativereason', ascending=False)

the most important reason for bad flight is due to a customer service… not so good for the companie

# Dividing Data into Training and Test Sets


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

# Training the Model


In [ ]:
 from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=190, random_state=0)
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

In [ ]:
predictions = text_classifier.predict(X_test)

Finally, to evaluate the performance of the machine learning models, we can use classification metrics such as a confusion metrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))